In [2]:
#Importamos las librerías necesarias
!git clone https://github.com/JaimeRosado/LegPy.git
import sys
#sys.path.insert(0,'/content/LegPy')
sys.path.insert(0,'LegPy')
import os
import numpy as np
import pandas as pd
import LegPy as lpy
Database = 'MIRD' # Lista de bases de datos disponibles: 'MIRD'
Isotopo = '177Lu' # Lista de isotopos disponibles: '99mTc', '111In', '161Tb', '177Lu' o '181Re'
Isotopos_con_espectro_continuo = ('188Re', '186Re' , '177Lu', '161Tb', '99mTc') # Isotopos de la lista de isotopos disponible que emiten fotones
medium = lpy.Medium(name='Water, Liquid') # Definimos en LegPy la composición química del medio sobre el que se va a hacer la simulación. En este caso partícular es agua
diameter_list = [10000, 5000, 2000, 1000, 500, 200, 100, 50, 20, 10] # Lista con los diámetros de las esferas usadas en la referencia
# Función donde cargamos los espectros de emisión de fotones, procedente de un archivo .txt con los datos.
def cargar_txt(nombre_archivo):
    if not os.path.exists(nombre_archivo):
        raise FileNotFoundError(f'Archivo no encontrado: {nombre_archivo}')
    return np.loadtxt(nombre_archivo)

nombre_archivo = Isotopo + '_' + Database + '_Fotones.txt' # Definimos el nombre del archivo .txt con el espectro
E_w = cargar_txt(nombre_archivo) # Cargamos el espectro
E_w[:,0] = E_w[:,0] / 1000 # Convertimos las energías a MeV
normalizacion = E_w[:,1].sum() / 100 # Calculamos el número de fotones por desintegración
espectros = lpy.Spectrum(name='multi_mono', E_w=E_w)

# Función para realizar la simulación a partir de las mismas condiciones que las de la referencia. La función pide el diámetro de la esfera (en micrómetros), la composición química del medio, el
# número de fotones emitidos por desintegración y el número de partículas a simular
def simular_dosis(d, medium, espectros, normalizacion, n_part=2000000, rho = 1e-3, n_r = 3000):
    R_um = d / 2 # Radio de la esfera, en micrómetros
    R = R_um / 1e4 # Radio de la esfera, en centímetros
    V_um3 = (4/3) * np.pi * R_um**3 # Volumen de la esfera en micrómetros cúbicos
    geometry = lpy.Geometry(name='sphere', r=R, n_r=n_r) # Definimos en LegPy la geometría del medio. En este caso partícular es una esfera
    beam = lpy.Beam(name='isotropic', particle='photon', p_in=[0., 0., 0.], r_s=R) # Definimos en LegPy la geometría del haz de partículas. En este caso se trata de una distribución
    # uniforme a lo largo de la esfera de fuentes puntuales
    Edep = result = lpy.MC(medium, geometry, espectros, beam , n_part = n_part).Edep # Simulación en Monte Carlo en las condiciones establecidas
    Vp = geometry.delta_v # Volumen de cada bin de la esfera
    E_bins = Vp*Edep # Multiplicamos la energía depositada por volumen en cada bin por el volumen de cada bin para obtener la energía depositada en cada bin de la esfera, en keV
    E_dep = E_bins.sum() # Sumamos todas las contribuciones para obtener la energía depositada en la esfera, en keV
    dose_e = E_dep*1.602e-16 / (rho * (4/3) * np.pi * R**3 ) # Dividimos la energía depositada, en J, entre la masa de la esfera, en kg, para obtener la dosis depositada, en Gy
    dose_um3 = dose_e * V_um3 # Multiplicamos por el volumen para obtener la dosis depositada por los fotones para una desintegración por micrómetro cúbico
    print('Diámetro de la esfera = ', d, ' micrómetros')
    print('Dosis depositada por los fotones para una desintegración por micrómetro cúbico: ', dose_um3, ' Gy')
    print('---')
    return [2 * R_um, dose_um3 * normalizacion] # Devolvemos una lista con dos valores: El diámetro de la esfera y la dosis depositada por los fotones para una desintegración por micrómetro cúbico

print('Se va a calcular la dosis depositada por los fotones para una distribución uniforme de 1 desintegración por micrómetro cúbico del isótopo ',Isotopo, ' a partir de la base de datos ', Database)
print('Datos de la simuación:')
print('Número de fotones emitidos por desintegración: ', normalizacion)
print('---')
resultados = [simular_dosis(d, medium, espectros, normalizacion) for d in diameter_list] # Se llama a la función para realizar la simulación

fatal: destination path 'LegPy' already exists and is not an empty directory.
Se va a calcular la dosis depositada por los fotones para una distribución uniforme de 1 desintegración por micrómetro cúbico del isótopo  177Lu  a partir de la base de datos  MIRD
Datos de la simuación:
Número de fotones emitidos por desintegración:  0.426238
---

The simulation has ended

Computing time per beam particle =  7.18e-05 seconds

Diámetro de la esfera =  10000  micrómetros
Dosis depositada por los fotones para una desintegración por micrómetro cúbico:  0.5647139293029247  Gy
---

The simulation has ended

Computing time per beam particle =  6.90e-05 seconds

Diámetro de la esfera =  5000  micrómetros
Dosis depositada por los fotones para una desintegración por micrómetro cúbico:  0.4827858109637943  Gy
---

The simulation has ended

Computing time per beam particle =  6.62e-05 seconds

Diámetro de la esfera =  2000  micrómetros
Dosis depositada por los fotones para una desintegración por micróme